In [1]:
from thesis_library import *

# Previous day values

# Random starting values

In [2]:
theta1 = optimize_on_day_with_starting_values(date_number=0, method='BFGS', theta0=np.array([2, 5, 2, 6, 1, 4]))

/Users/andrey_lukyanov/Google_Drive/Studies/Year_4/Курсач/Coding/Comparing-numerical-methods-for-term-structure-fitting/thesis_library.py:103: OptimizeWarning: Unknown solver options: xtol
  res = minimize(loss_func, theta0, method='BFGS', options={'xtol': 1e-8, 'disp': False, 'maxiter': 100000})


# Diebold, Li starting values